## Convert DOZ files to structured parquets to ease processing for the file reader

In [1]:
from concurrent.futures import ThreadPoolExecutor
from doz_file_utils import compose_doz, parse_doz
from doz_file_utils.doz_dataclasses import ParsedDOZ
import glob
import numpy as np
import os
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from tqdm import tqdm

from parse_data import _parse_single_file

In [2]:
sub_res = {}
for subject in glob.glob("../corr/train/raw/*"):
    all_paths = glob.glob(os.path.join(subject, "**/*"), recursive=True)
    file_paths = [p for p in all_paths if os.path.isfile(p)]
    results = []

    with ThreadPoolExecutor() as executor:
        futures = executor.map(_parse_single_file, file_paths)

        for parsed in tqdm(futures, desc="Parsing DOZ files", total=len(file_paths)):
            if parsed is not None:
                results.append(parsed)
    
    df = pd.DataFrame(results, columns=["ts", "gain", "signals"])
    table = pa.Table.from_pandas(df, preserve_index=False)
    pq.write_table(table, f"data/{subject.split('/')[4]}.parquet")

Parsing DOZ files: 100%|██████████| 2006/2006 [00:01<00:00, 1453.63it/s]
